교차 검증 : 본고사가 테스트 데이터 세트에 대해 평가하는 거라면 모의고사는 교차 검증에서 많은 학습과 검증 세트에서 알고리즘 학습과 평가를 수행하는 것

별도의 여러 세트로 구성된 학습 데이터 세트와 검증 데이터 세트에서 학습과 평가를 수행


# KFold

<br>
k폴드 교차 검증

- k개의 데이터 폴드 세트를 만들어 k번만큼 각 폴드 세트에 학습과 검증 평가를 반복적으로 수행

- k개의 폴드된 데이터 세트를 학습과 검증을 위한 데이터 세트로 변경하면서 k번 평가를 수행한 뒤 이 k개의 평가를 평균한 결과를 갖고 예측 성능을 평가

<br>
KFold클래스를 이용해 붓꽃 데이터 세트를 교차 검증하고 예측 정확도 확인

붓꽃 데이터 세트와 DecisionTreeClassifier를 다시 생성 후 5개의 폴드 세트로 분리하는 KFold 객체를 생성

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

# 데이터 불러오기
iris = load_iris()
print(iris.keys())
features = iris.data
target = iris.target
print('\n붓꽃 데이터 세트 크기 : ', features.shape[0])

# 모델 객체 생성
dt = DecisionTreeClassifier(random_state = 156)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

붓꽃 데이터 세트 크기 :  150


In [2]:
# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성
kfold = KFold(n_splits = 5)
cv_accuracy = []

생성된 KFold 객체의 split()을 호출해 전체 붓꽃 데이터를 5개의 폴드 데이터 세트로 분리

KFold 객체는 split()을 호출하면 학습용/검증용 데이터로 분할할 수 있는 인덱스를 반환

다음은 5개의 폴드 세트를 생성하는 KFold객체의 split()을 호출해 교차 검증 수행 시마다 학습과 검증을 반복해 예측 정확도를 측정

In [3]:
n_iter = 0

# KFold 객체의 split()를 호출하면 폴드별 학습용, 검증용 테스트의 로우 인덱슬을 array로 반환
# 150개 데이터 중 학습용 데이터는 120개 검증 테스트 데이터는 30개
for train_index, test_index in kfold.split(features):
    # kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    # 학습 및 예측
    dt.fit(X_train, y_train)
    pred = dt.predict(X_test)
    n_iter += 1
    
    # 반복 시마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n{}교차 검증 정확도 : {}, 학습 데이터 크기 : {}, 검증 데이터 크기 : {}'.format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n평균 검증 정확도 : {}'.format(np.mean(cv_accuracy)))


1교차 검증 정확도 : 1.0, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30

2교차 검증 정확도 : 0.9667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30

3교차 검증 정확도 : 0.8667, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30

4교차 검증 정확도 : 0.9333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30

5교차 검증 정확도 : 0.7333, 학습 데이터 크기 : 120, 검증 데이터 크기 : 30

평균 검증 정확도 : 0.9


# Stratified K 폴드

불균형한 분포도를 가진 레이블 데이터 집합을 위한 k폴드 방식

불균형한 분포도 - 특정 레이블 값이 특이하게 많거나 매우 적어서 값의 분포가 한쪽으로 치우치는 것을 의미한다

<br>

원본 데이터의 레이블 분포를 먼저 고려한 뒤 이 분포와 동일하게 학습과 검증 데이터 세트를 분배

In [4]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target

iris_df['label'].value_counts()

label
0    50
1    50
2    50
Name: count, dtype: int64

3개의 폴드 세트를 KFold로 생성하고, 각 교차 검증 시마다 생성되는 학습/검증 레이블 데이터 값의 분포도 확인

In [5]:
iris_df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [6]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 3)
n_iter = 0

for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n교차검증 : {}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('ln\\검증 레이블 데이터 분포 : \n', label_test.value_counts())    


교차검증 : 1
학습 레이블 데이터 분포 : 
 label
1    50
2    50
Name: count, dtype: int64
ln\검증 레이블 데이터 분포 : 
 label
0    50
Name: count, dtype: int64

교차검증 : 2
학습 레이블 데이터 분포 : 
 label
0    50
2    50
Name: count, dtype: int64
ln\검증 레이블 데이터 분포 : 
 label
1    50
Name: count, dtype: int64

교차검증 : 3
학습 레이블 데이터 분포 : 
 label
0    50
1    50
Name: count, dtype: int64
ln\검증 레이블 데이터 분포 : 
 label
2    50
Name: count, dtype: int64


=> 교차 검증 시마다 3개의 폴드 세트로 만들어지는 학습 레이블과 검증 레이블이 완전히 다른 값으로 추출

In [7]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits = 3)
n_iter = 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('\n교차검증 : {}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n', label_train.value_counts())
    print('ln\\검증 레이블 데이터 분포 : \n', label_test.value_counts())   


교차검증 : 1
학습 레이블 데이터 분포 : 
 label
2    34
0    33
1    33
Name: count, dtype: int64
ln\검증 레이블 데이터 분포 : 
 label
0    17
1    17
2    16
Name: count, dtype: int64

교차검증 : 2
학습 레이블 데이터 분포 : 
 label
1    34
0    33
2    33
Name: count, dtype: int64
ln\검증 레이블 데이터 분포 : 
 label
0    17
2    17
1    16
Name: count, dtype: int64

교차검증 : 3
학습 레이블 데이터 분포 : 
 label
0    34
1    33
2    33
Name: count, dtype: int64
ln\검증 레이블 데이터 분포 : 
 label
1    17
2    17
0    16
Name: count, dtype: int64


StratifiedFKold를 이용해 붓꽃 데이터를 교차 검증

In [8]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 156)

skf = StratifiedKFold(n_splits = 3)
n_iter = 0
cv_accuracy = []

features = np.array(iris_df.iloc[:, 0:4])
target = np.array(iris_df['label'])

for train_index, test_index in skf.split(features, target):
    # split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    # 학습 및 예측
    dt.fit(X_train, y_train)
    pred = dt.predict(X_test)
    
    # 반복 시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n# {}교차 검증 정확도 : {}, 학습 데이터 크기 : {}, 검증 데이터 크기 : {}'.format(n_iter, accuracy, train_size, test_size))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도 : ', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도 : ', np.round(np.mean(cv_accuracy), 4))


# 1교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50

# 2교차 검증 정확도 : 0.94, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50

# 3교차 검증 정확도 : 0.98, 학습 데이터 크기 : 100, 검증 데이터 크기 : 50

## 교차 검증별 정확도 :  [0.98 0.94 0.98]
## 평균 검증 정확도 :  0.9667


=> 일반적으로 분류에서의 교차 검증은 K폴드가 아닌 Stratified K폴드로 분할돼야 한다

회귀에서는 Stratified K 폴드가 지원되지 않는다 -> 회귀의 결정값은 이산값 형태의 레이블이 아니라 연속된 숫자값이기 때문이다

# 사이킷런

교차 검증을 보다 간편하게 - cross_val_score()

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt = DecisionTreeClassifier(random_state = 156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도, 교차 검증 세트는 3개
scores = cross_val_score(dt, data, label, scoring = 'accuracy', cv = 3)
print('교차 검증별 정확도 : ', np.round(scores, 4))
print('평균 검증 정확도 : ', np.round(np.mean(scores), 4))

교차 검증별 정확도 :  [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667
